# Data Preparation

## Initialize Modules and Configurations

In [1]:
# Import required modules
import pandas as pd
import numpy as np
import pickle

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

## Import Data

In [2]:
# Import training data set
train = pd.read_csv("../data/raw/train.csv.zip", compression="zip", index_col='id')

# Import metadata create from exploratory stage
meta_data = pd.read_csv("../references/training_metadata.csv")

# Import variables to drop determined in exploratory
cont_names_to_drop = pickle.load(open("../data/interim/cont_names_to_drop.p", "rb" ))

## Establish Variables from Metadata

In [ ]:
# Seperate Continious and Categorical Attributes
cat_names = meta_data[meta_data['Data Type']=='object']['Variable']
cont_names = meta_data[(meta_data['Data Type']=='float64') & 
                       (meta_data['Variable']!='loss')]['Variable']

cont_names_final = cont_names[~cont_names.isin(cont_names_to_drop)]

# Binary Encoded Feature Dataset

In [ ]:
import category_encoders as ce
encoder = ce.BinaryEncoder()

categorical_data = encoder.fit_transform(train[cat_names])
continuous_data = train[cont_names]

train_binary = pd.concat([train[cont_names], categorical_data], axis=1)
#train_binary.to_csv("../data/processed/train_binary_encoded.csv", index=False)
train_binary.to_pickle("../data/processed/train_binary_encoded.p")